In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the cleaned data
data = pd.read_csv('/content/drive/MyDrive/data_cleaned.csv')

# Separate features and target
X = data.drop(columns=['MEDV'])
y = data['MEDV']

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert data to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32).view(-1, 1)

# Create TensorDataset and DataLoader
dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

In [13]:
from sklearn.ensemble import RandomForestRegressor

# Train FCNN
class FCNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model_fcnn = FCNN(input_dim, hidden_dim, output_dim)
optimizer_fcnn = torch.optim.Adam(model_fcnn.parameters(), lr=0.001)
criterion_fcnn = nn.MSELoss()

def train_fcnn():
    model_fcnn.train()
    for data, target in train_loader:
        optimizer_fcnn.zero_grad()
        output = model_fcnn(data)
        loss = criterion_fcnn(output, target)
        loss.backward()
        optimizer_fcnn.step()



In [15]:
for epoch in range(101, 400):
    train_fcnn()

# Train RandomForest
rf_model = RandomForestRegressor(n_estimators=100, random_state=13)
rf_model.fit(X, y)

# Ensemble Predictions
def ensemble_predict(data):
    fcnn_preds = model_fcnn(data).detach().numpy()
    rf_preds = rf_model.predict(data)
    return (fcnn_preds + rf_preds) / 2

# Evaluate Ensemble Model
def test_ensemble(loader):
    model_fcnn.eval()
    losses = []
    with torch.no_grad():
        for data, target in loader:
            preds = ensemble_predict(data)
            loss = criterion_fcnn(torch.tensor(preds, dtype=torch.float32), target)
            losses.append(loss.item())
    return np.mean(losses)

In [16]:
train_losses = []
test_losses = []

for epoch in range(400):
    train_fcnn()
    train_loss = test_ensemble(train_loader)
    test_loss = test_ensemble(test_loader)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    print(f'Epoch: {epoch+1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse

Epoch: 1, Train Loss: 38.4049, Test Loss: 80.9344


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse

Epoch: 2, Train Loss: 43.3629, Test Loss: 83.4652
Epoch: 3, Train Loss: 43.7231, Test Loss: 83.4627


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse

...

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse

Epoch: 396, Train Loss: 40.1066, Test Loss: 82.0038
Epoch: 397, Train Loss: 39.7150, Test Loss: 82.7507
Epoch: 398, Train Loss: 40.4184, Test Loss: 81.7715


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse

Epoch: 399, Train Loss: 39.7130, Test Loss: 81.4322
Epoch: 400, Train Loss: 40.7383, Test Loss: 81.6291


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse

In [17]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0]
}

# Create a model
xgb_model = XGBRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X, y)

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", -grid_search.best_score_)


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters found:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.9}
Best cross-validation score:  29.075743202110374


In [18]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Define models to evaluate
models = {
    'Ridge': Ridge(),
    'RandomForest': RandomForestRegressor(),
    'SVR': SVR(),
    'XGBRegressor': XGBRegressor()
}

# Evaluate each model using cross-validation
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    print(f"{name} - Mean MSE: {-scores.mean()}, Std: {scores.std()}")


Ridge - Mean MSE: 66.06997533975634, Std: 49.84965359442092
RandomForest - Mean MSE: 31.193373888971337, Std: 19.92650290439564
SVR - Mean MSE: 51.89736816291925, Std: 22.45709003617703
XGBRegressor - Mean MSE: 37.833475162836365, Std: 17.535910183901613


In [22]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 3.0 MB/s eta 0:00:00


In [25]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

# Define models
xgb_model = XGBRegressor(n_estimators=300, learning_rate=0.1, max_depth=5, subsample=0.9)
lgbm_model = LGBMRegressor(n_estimators=300, learning_rate=0.1, max_depth=5, subsample=0.9, num_leaves=32)
catboost_model = CatBoostRegressor(n_estimators=300, learning_rate=0.1, max_depth=5, subsample=0.9, verbose=0)

# Fit the models
xgb_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)
catboost_model.fit(X_train, y_train)

# Make predictions
xgb_preds = xgb_model.predict(X_test)
lgbm_preds = lgbm_model.predict(X_test)
catboost_preds = catboost_model.predict(X_test)

# Calculate MSE
xgb_mse = mean_squared_error(y_test, xgb_preds)
lgbm_mse = mean_squared_error(y_test, lgbm_preds)
catboost_mse = mean_squared_error(y_test, catboost_preds)

print(f"XGBoost MSE: {xgb_mse}")
print(f"LightGBM MSE: {lgbm_mse}")
print(f"CatBoost MSE: {catboost_mse}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000076 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 914
[LightGBM] [Info] Number of data points in the train set: 392, number of used features: 12
[LightGBM] [Info] Start training from score 23.046939
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [26]:
# Define and train multiple neural networks
networks = [FCNN(input_dim, hidden_dim, output_dim) for _ in range(5)]
optimizers = [torch.optim.Adam(net.parameters(), lr=0.001) for net in networks]

for epoch in range(200):
    for net, optimizer in zip(networks, optimizers):
        net.train()
        for data, target in train_loader:
            optimizer.zero_grad()
            output = net(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

# Combine predictions
def ensemble_predict(data):
    preds = [net(data).detach().numpy() for net in networks]
    return np.mean(preds, axis=0)

# Evaluate ensemble model
def test_ensemble(loader):
    losses = []
    with torch.no_grad():
        for data, target in loader:
            preds = ensemble_predict(data)
            loss = criterion(torch.tensor(preds, dtype=torch.float32), target)
            losses.append(loss.item())
    return np.mean(losses)

train_loss = test_ensemble(train_loader)
test_loss = test_ensemble(test_loader)
print(f"Train Loss: {train_loss}, Test Loss: {test_loss}")


Train Loss: 6.940478269870464, Test Loss: 17.662588953971863


In [27]:
from sklearn.decomposition import PCA

# Add new features (e.g., interaction terms, polynomial features)
X['CRIM*ZN'] = X['CRIM'] * X['ZN']
X['LSTAT^2'] = X['LSTAT'] ** 2

# Apply PCA for dimensionality reduction
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

# Train a model on the transformed features
model = XGBRegressor()
model.fit(X_pca, y)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [28]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Create the model
xgb_model = XGBRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", -grid_search.best_score_)


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best parameters found:  {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Best cross-validation score:  19.161750949994797


In [29]:
from sklearn.ensemble import VotingRegressor

# Create individual models
xgb_model = XGBRegressor(**grid_search.best_params_)
lgbm_model = LGBMRegressor(n_estimators=300, learning_rate=0.1, max_depth=5, subsample=0.9, num_leaves=32)
catboost_model = CatBoostRegressor(n_estimators=300, learning_rate=0.1, max_depth=5, subsample=0.9, verbose=0)

# Create an ensemble of models
ensemble = VotingRegressor(estimators=[
    ('xgb', xgb_model),
    ('lgbm', lgbm_model),
    ('cat', catboost_model)
])

# Fit the ensemble model
ensemble.fit(X_train, y_train)

# Make predictions
ensemble_preds = ensemble.predict(X_test)

# Calculate MSE
ensemble_mse = mean_squared_error(y_test, ensemble_preds)

print(f"Ensemble MSE: {ensemble_mse}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000077 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 914
[LightGBM] [Info] Number of data points in the train set: 392, number of used features: 12
[LightGBM] [Info] Start training from score 23.046939
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [33]:
# Load the cleaned data
data = pd.read_csv('/content/drive/MyDrive/data_cleaned.csv')

# Separate features and target
X = data.drop(columns=['MEDV'])
y = data['MEDV']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Add interaction terms or polynomial features
X_df = pd.DataFrame(X_scaled, columns=X.columns)
X_df['CRIM*ZN'] = X_df['CRIM'] * X_df['ZN']
X_df['LSTAT^2'] = X_df['LSTAT'] ** 2

# Apply PCA for dimensionality reduction
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_df)

# Split the transformed data into training and testing sets
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Create the model
xgb_model = XGBRegressor()

# Create the GridSearchCV object
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train_pca, y_train_pca)

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", -grid_search.best_score_)

# Fit a model on the transformed data using the best parameters
xgb_model_pca = XGBRegressor(**grid_search.best_params_)
xgb_model_pca.fit(X_train_pca, y_train_pca)
xgb_preds_pca = xgb_model_pca.predict(X_test_pca)
xgb_mse_pca = mean_squared_error(y_test_pca, xgb_preds_pca)

print(f"XGBoost with PCA MSE: {xgb_mse_pca}")

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best parameters found:  {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'subsample': 0.8}
Best cross-validation score:  28.475993821422236
XGBoost with PCA MSE: 28.09603576155629


In [34]:
xgb_model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.8,
    reg_alpha=0.1,  # L1 regularization
    reg_lambda=0.1,  # L2 regularization
)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],
    eval_metric='rmse',
    early_stopping_rounds=50,
    verbose=True
)

xgb_preds = xgb_model.predict(X_test)
xgb_mse = mean_squared_error(y_test, xgb_preds)

print(f"XGBoost with Regularization and Early Stopping MSE: {xgb_mse}")


[0]	validation_0-rmse:9.62155	validation_1-rmse:8.98676
[1]	validation_0-rmse:9.53911	validation_1-rmse:8.90895
[2]	validation_0-rmse:9.45742	validation_1-rmse:8.83943
[3]	validation_0-rmse:9.37861	validation_1-rmse:8.77141
[4]	validation_0-rmse:9.29929	validation_1-rmse:8.69697
[5]	validation_0-rmse:9.21868	validation_1-rmse:8.61801
[6]	validation_0-rmse:9.13752	validation_1-rmse:8.54676
[7]	validation_0-rmse:9.05646	validation_1-rmse:8.48452
[8]	validation_0-rmse:8.97923	validation_1-rmse:8.41939
[9]	validation_0-rmse:8.90646	validation_1-rmse:8.35844
[10]	validation_0-rmse:8.82910	validation_1-rmse:8.29448
[11]	validation_0-rmse:8.75498	validation_1-rmse:8.22530
[12]	validation_0-rmse:8.68011	validation_1-rmse:8.15233
[13]	validation_0-rmse:8.60977	validation_1-rmse:8.09550
[14]	validation_0-rmse:8.53730	validation_1-rmse:8.03239
[15]	validation_0-rmse:8.46896	validation_1-rmse:7.96213
[16]	validation_0-rmse:8.40106	validation_1-rmse:7.92265
[17]	validation_0-rmse:8.33319	validation

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[31]	validation_0-rmse:7.42311	validation_1-rmse:7.12345
[32]	validation_0-rmse:7.36087	validation_1-rmse:7.07551
[33]	validation_0-rmse:7.30063	validation_1-rmse:7.02456
[34]	validation_0-rmse:7.24476	validation_1-rmse:6.97878
[35]	validation_0-rmse:7.18512	validation_1-rmse:6.94267
[36]	validation_0-rmse:7.12757	validation_1-rmse:6.88848
[37]	validation_0-rmse:7.07026	validation_1-rmse:6.83555
[38]	validation_0-rmse:7.01205	validation_1-rmse:6.77963
[39]	validation_0-rmse:6.95384	validation_1-rmse:6.73560
[40]	validation_0-rmse:6.89998	validation_1-rmse:6.69457
[41]	validation_0-rmse:6.84849	validation_1-rmse:6.64131
[42]	validation_0-rmse:6.79343	validation_1-rmse:6.58742
[43]	validation_0-rmse:6.73925	validation_1-rmse:6.54398
[44]	validation_0-rmse:6.68683	validation_1-rmse:6.49500
[45]	validation_0-rmse:6.63212	validation_1-rmse:6.44855
[46]	validation_0-rmse:6.59212	validation_1-rmse:6.42313
[47]	validation_0-rmse:6.54075	validation_1-rmse:6.38144
[48]	validation_0-rmse:6.48688	

In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# Train a simple linear regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
linear_preds = linear_model.predict(X_test)
linear_mse = mean_squared_error(y_test, linear_preds)

# Train a simple decision tree regressor
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)
tree_preds = tree_model.predict(X_test)
tree_mse = mean_squared_error(y_test, tree_preds)

print(f"Linear Regression MSE: {linear_mse}")
print(f"Decision Tree Regressor MSE: {tree_mse}")


Linear Regression MSE: 36.74371777524715
Decision Tree Regressor MSE: 25.769696969696973


In [36]:
from xgboost import XGBRegressor
import joblib

# Define and train the XGBoost model
xgb_model = XGBRegressor(
    colsample_bytree=0.8,
    learning_rate=0.01,
    max_depth=5,
    n_estimators=1000,
    subsample=0.9,
    reg_alpha=0.1,
    reg_lambda=0.1
)

xgb_model.fit(X_train, y_train, early_stopping_rounds=100, eval_set=[(X_test, y_test)], verbose=True)
# Save the model to a file
joblib.dump(xgb_model, 'xgboost_model.pkl')
print("Model saved to xgboost_model.pkl")


[0]	validation_0-rmse:8.98676
[1]	validation_0-rmse:8.90895
[2]	validation_0-rmse:8.83943
[3]	validation_0-rmse:8.77141
[4]	validation_0-rmse:8.69697
[5]	validation_0-rmse:8.61801
[6]	validation_0-rmse:8.54676
[7]	validation_0-rmse:8.48452
[8]	validation_0-rmse:8.41939
[9]	validation_0-rmse:8.35844
[10]	validation_0-rmse:8.29448
[11]	validation_0-rmse:8.22530
[12]	validation_0-rmse:8.15233
[13]	validation_0-rmse:8.09550
[14]	validation_0-rmse:8.03239
[15]	validation_0-rmse:7.96213
[16]	validation_0-rmse:7.92265
[17]	validation_0-rmse:7.86397
[18]	validation_0-rmse:7.81638
[19]	validation_0-rmse:7.75824
[20]	validation_0-rmse:7.69532
[21]	validation_0-rmse:7.63288
[22]	validation_0-rmse:7.57068
[23]	validation_0-rmse:7.51727
[24]	validation_0-rmse:7.45454
[25]	validation_0-rmse:7.40157
[26]	validation_0-rmse:7.36502
[27]	validation_0-rmse:7.31720
[28]	validation_0-rmse:7.25261
[29]	validation_0-rmse:7.20849
[30]	validation_0-rmse:7.16445
[31]	validation_0-rmse:7.12345
[32]	validation_0-

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[59]	validation_0-rmse:5.88600
[60]	validation_0-rmse:5.84604
[61]	validation_0-rmse:5.81125
[62]	validation_0-rmse:5.77351
[63]	validation_0-rmse:5.75126
[64]	validation_0-rmse:5.71080
[65]	validation_0-rmse:5.67898
[66]	validation_0-rmse:5.63934
[67]	validation_0-rmse:5.61471
[68]	validation_0-rmse:5.57809
[69]	validation_0-rmse:5.55512
[70]	validation_0-rmse:5.51909
[71]	validation_0-rmse:5.49369
[72]	validation_0-rmse:5.46400
[73]	validation_0-rmse:5.42798
[74]	validation_0-rmse:5.39545
[75]	validation_0-rmse:5.35974
[76]	validation_0-rmse:5.32784
[77]	validation_0-rmse:5.30820
[78]	validation_0-rmse:5.28862
[79]	validation_0-rmse:5.26457
[80]	validation_0-rmse:5.23060
[81]	validation_0-rmse:5.19675
[82]	validation_0-rmse:5.16624
[83]	validation_0-rmse:5.13121
[84]	validation_0-rmse:5.10463
[85]	validation_0-rmse:5.07390
[86]	validation_0-rmse:5.04218
[87]	validation_0-rmse:5.01685
[88]	validation_0-rmse:4.99322
[89]	validation_0-rmse:4.96507
[90]	validation_0-rmse:4.93984
[91]	val

In [37]:
# Load the model from the file
xgb_model_loaded = joblib.load('xgboost_model.pkl')
print("Model loaded from xgboost_model.pkl")


Model loaded from xgboost_model.pkl


In [38]:
# Make predictions with the loaded model
predictions = xgb_model_loaded.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f"Loaded model Test MSE: {mse}")


Loaded model Test MSE: 8.392364933935434


In [39]:
# Example of using the model for new data
new_data = pd.read_csv('path_to_new_data.csv')
new_data_scaled = scaler.transform(new_data)   # Scale new data using the same scaler

# Load the model
xgb_model_loaded = joblib.load('xgboost_model.pkl')

# Make predictions on the new data
new_predictions = xgb_model_loaded.predict(new_data_scaled)
print("Predictions on new data:", new_predictions)


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_new_data.csv'

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from joblib import dump, load
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

# Load the data
housing = pd.read_csv('/content/drive/MyDrive/data_cleaned.csv')

# Split the data into training and testing sets using stratified sampling based on the 'CHAS' column
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['CHAS']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# Create a copy of the training data for exploration
housing = strat_train_set.copy()

# Create new attributes for feature engineering
housing["CRIM_PER_PT"] = housing['CRIM'] / housing['PTRATIO']
housing["AGE_BIN"] = np.floor(housing["AGE"] / 10) * 10

# Prepare the data for machine learning algorithms
housing = strat_train_set.drop("MEDV", axis=1)
housing_labels = strat_train_set["MEDV"].copy()

# Data preprocessing pipeline for numerical attributes
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

# Data preprocessing pipeline for categorical attributes
cat_pipeline = Pipeline([
    ('onehot', OneHotEncoder()),
])

# Combine numerical and categorical pipelines
num_attribs = list(housing.drop(['CHAS'], axis=1).columns)
cat_attribs = ['CHAS']

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

housing_prepared = full_pipeline.fit_transform(housing)

# Define the models
rf_model = RandomForestRegressor(random_state=42)
xgb_model = XGBRegressor(random_state=42)

# Train the Random Forest model
rf_model.fit(housing_prepared, housing_labels)

# Train the XGBoost model
xgb_model.fit(housing_prepared, housing_labels)

# Evaluate the Random Forest model using cross-validation
rf_scores = cross_val_score(rf_model, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
rf_rmse_scores = np.sqrt(-rf_scores)

# Evaluate the XGBoost model using cross-validation
xgb_scores = cross_val_score(xgb_model, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
xgb_rmse_scores = np.sqrt(-xgb_scores)

def print_scores(model_name, scores):
    print(f"{model_name} Scores:", scores)
    print(f"{model_name} Mean: ", scores.mean())
    print(f"{model_name} Standard deviation: ", scores.std())

print_scores("Random Forest", rf_rmse_scores)
print_scores("XGBoost", xgb_rmse_scores)

# Save the models
dump(rf_model, 'rf_housing_model.joblib')
dump(xgb_model, 'xgb_housing_model.joblib')
print("Models saved to rf_housing_model.joblib and xgb_housing_model.joblib")

# Load the models
rf_model_loaded = load('rf_housing_model.joblib')
xgb_model_loaded = load('xgb_housing_model.joblib')
print("Models loaded from rf_housing_model.joblib and xgb_housing_model.joblib")

# Prepare the test set
X_test = strat_test_set.drop("MEDV", axis=1)
Y_test = strat_test_set["MEDV"].copy()
X_test_prepared = full_pipeline.transform(X_test)

# Evaluate the Random Forest model on the test set
rf_final_predictions = rf_model_loaded.predict(X_test_prepared)
rf_final_mse = mean_squared_error(Y_test, rf_final_predictions)
rf_final_rmse = np.sqrt(rf_final_mse)
print(f"Random Forest Final RMSE on the test set: {rf_final_rmse}")

# Evaluate the XGBoost model on the test set
xgb_final_predictions = xgb_model_loaded.predict(X_test_prepared)
xgb_final_mse = mean_squared_error(Y_test, xgb_final_predictions)
xgb_final_rmse = np.sqrt(xgb_final_mse)
print(f"XGBoost Final RMSE on the test set: {xgb_final_rmse}")


Random Forest Scores: [4.70131959 5.37819324 2.53880605 3.16603768 5.10342101 5.61596865
 2.89497432 2.79676277 3.53836665 3.91031402]
Random Forest Mean:  3.964416397613944
Random Forest Standard deviation:  1.0921578591973808
XGBoost Scores: [5.62262948 6.74368432 2.72090821 4.01210315 5.05194837 7.30687908
 4.11483896 2.72731979 4.07106134 2.84766596]
XGBoost Mean:  4.521903867250171
XGBoost Standard deviation:  1.5518049782728827
Models saved to rf_housing_model.joblib and xgb_housing_model.joblib
Models loaded from rf_housing_model.joblib and xgb_housing_model.joblib
Random Forest Final RMSE on the test set: 3.637359513757642
XGBoost Final RMSE on the test set: 3.778370448436331
Random Forest Prediction for new data: [41.085]
XGBoost Prediction for new data: [47.946377]


In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from joblib import dump, load
import matplotlib.pyplot as plt

# Load the data
housing = pd.read_csv('/content/drive/MyDrive/data_cleaned.csv')

# Verify the columns in the dataset
print("Columns in dataset:", housing.columns)

# Split the data into training and testing sets using stratified sampling based on the 'CHAS' column
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['CHAS']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# Create a copy of the training data for exploration
housing = strat_train_set.copy()

# Verify the columns in the training dataset
print("Columns in training set:", housing.columns)

Columns in dataset: Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')
Columns in training set: Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')
